In [2]:
import keras
import keras.applications
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from PIL import Image
import os
import gc

### Reading in Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
manual_label = pd.read_excel("/content/drive/MyDrive/Data403Materials/Data403Project3/Copy of Manual Labeling.xlsx")
manual_label.head()

,Image,Old,More than 1 person,Saturation (Strong/Weak),Nature,Blurry,Castle,Desert,Filter/Not,Europe,Straight,Shadows (Strong/Weak)
0,Project3\data403-project3\Training_Data\Alex\A...,1,0,1,1,1,0,0,0,0,1,0
1,Project3\data403-project3\Training_Data\Alex\A...,1,0,1,1,1,0,0,1,1,0,1
2,Project3\data403-project3\Training_Data\Alex\A...,1,0,0,0,1,0,0,0,0,0,0
3,Project3\data403-project3\Training_Data\Alex\A...,0,0,1,1,0,0,0,1,0,0,0
4,Project3\data403-project3\Training_Data\Alex\A...,0,0,1,1,0,0,0,0,0,1,1


In [5]:
manual_label["Image"] = manual_label["Image"].str.split("\\").str[-1]

In [6]:
alex = manual_label[manual_label["Image"].str.contains("Alex")]
alex["Target"] = 1
kelly = manual_label[manual_label["Image"].str.contains("Kelly")]
kelly["Target"] = 0

<ipython-input-6-095b37fda2ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alex["Target"] = 1
<ipython-input-6-095b37fda2ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kelly["Target"] = 0


In [7]:
info_in_order = pd.concat([alex, kelly], axis = 0).iloc[:, 1:]
info_in_order

,Old,More than 1 person,Saturation (Strong/Weak),Nature,Blurry,Castle,Desert,Filter/Not,Europe,Straight,Shadows (Strong/Weak),Target
0,1,0,1,1,1,0,0,0,0,1,0,1
1,1,0,1,1,1,0,0,1,1,0,1,1
2,1,0,0,0,1,0,0,0,0,0,0,1
3,0,0,1,1,0,0,0,1,0,0,0,1
4,0,0,1,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
480,0,0,0,1,1,0,0,0,0,0,0,0
481,0,0,1,1,0,0,0,0,0,1,0,0
482,0,1,0,0,0,0,0,0,0,0,1,0
483,1,1,0,0,1,0,0,0,0,0,0,0


### Resnet

In [ ]:
set = keras.utils.image_dataset_from_directory(
    directory = "Training_Data",
    image_size = (224, 224),
    shuffle = False,
    batch_size = 485
)

In [ ]:
accuracy_list = []
accuracy_list_with_manual = []
def cross_val(num_folds = 5):
    total_indices = np.array(list(range(0, 485)))
    np.random.shuffle(total_indices)
    for i in range(0, num_folds):
        # print(i * (total_indices.size-1) // num_folds, (i + 1) * (total_indices.size-1) // num_folds)
        test_indices = total_indices[i * (total_indices.size-1) // num_folds : (i + 1) * (total_indices.size-1) // num_folds]
        train_indices = total_indices[np.isin(total_indices, test_indices, invert = True)] # for some reason setdiff1d sorts in order... facepalm

        train_x = set.as_numpy_iterator().next()[0][train_indices]
        train_y = set.as_numpy_iterator().next()[1][train_indices]
        test_x = set.as_numpy_iterator().next()[0][test_indices]
        test_y = set.as_numpy_iterator().next()[1][test_indices]

        train = tf.data.Dataset.from_tensor_slices((tf.constant(train_x), tf.constant(train_y))).batch(32)
        test = tf.data.Dataset.from_tensor_slices((tf.constant(test_x), tf.constant(test_y))).batch(32)

        model = keras.applications.ResNet50(
            include_top = False,
            input_shape = (224, 224, 3)
        )
        model.trainable = False
        final_model = keras.models.Sequential()
        final_model.add(model)
        final_model.add(keras.layers.GlobalAveragePooling2D())
        final_model.add(keras.layers.Dense(64, activation = "relu"))
        final_model.add(keras.layers.Dropout(0.5))
        final_model.add(keras.layers.Dense(16, activation = "relu"))
        final_model.add(keras.layers.Dropout(0.3))
        final_model.add(keras.layers.Dense(1, activation = "sigmoid"))

        final_model.compile(optimizer = "adam",
                    loss = "binary_crossentropy",
                    metrics = ["accuracy"])
        final_model.fit(train, epochs = 10)

        accuracy_list.append(final_model.evaluate(test)[1])

        # combine both results and use log reg
        model_predictions = final_model.predict(train_x)
        train_image = info_in_order.iloc[train_indices]
        actual_y = train_image.iloc[:, -1]
        intermediate = train_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop = True), pd.DataFrame(model_predictions, columns = ["prediction"])], axis = 1)

        log_reg = LogisticRegression()
        log_reg.fit(actual_x, actual_y)

        model_predictions = final_model.predict(test_x)
        test_image = info_in_order.iloc[test_indices]
        actual_y = test_image.iloc[:, -1]
        intermediate = test_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop = True), pd.DataFrame(model_predictions, columns = ["prediction"])], axis = 1)
        accuracy_list_with_manual.append(accuracy_score(actual_y, log_reg.predict(actual_x)))
        print(accuracy_score(actual_y, log_reg.predict(actual_x)))




cross_val(num_folds = 5)
print(accuracy_list)

Epoch 1/10
13/13 [==============================] - 5s 64ms/step - loss: 0.7380 - accuracy: 0.5604
Epoch 2/10
13/13 [==============================] - 1s 59ms/step - loss: 0.6221 - accuracy: 0.6581
Epoch 3/10
13/13 [==============================] - 1s 59ms/step - loss: 0.5171 - accuracy: 0.7275
Epoch 4/10
13/13 [==============================] - 1s 59ms/step - loss: 0.5114 - accuracy: 0.7378
Epoch 5/10
13/13 [==============================] - 1s 59ms/step - loss: 0.4802 - accuracy: 0.7789
Epoch 6/10
13/13 [==============================] - 1s 59ms/step - loss: 0.3923 - accuracy: 0.7969
Epoch 7/10
13/13 [==============================] - 1s 59ms/step - loss: 0.3944 - accuracy: 0.8355
Epoch 8/10
13/13 [==============================] - 1s 60ms/step - loss: 0.3408 - accuracy: 0.8458
Epoch 9/10
13/13 [==============================] - 1s 59ms/step - loss: 0.3466 - accuracy: 0.8406
Epoch 10/10
3/3 [==============================] - 0s 58ms/step
0.9895833333333334
Epoch 1/10
13/13 [========

CNN

In [ ]:
np.array(accuracy_list).mean()

0.8181915760040284

In [ ]:
np.array(accuracy_list_with_manual).mean()

0.9793599656357388

### Performance

In [ ]:
set = keras.utils.image_dataset_from_directory(
    directory = "Training_Data",
    image_size = (224, 224),
    shuffle = True,
    batch_size = 32
)
model = keras.applications.ResNet50(
    include_top = False,
    input_shape = (224, 224, 3)
)
model.trainable = False
final_model = keras.models.Sequential()
final_model.add(model)
final_model.add(keras.layers.GlobalAveragePooling2D())
final_model.add(keras.layers.Dense(64, activation = "relu"))
final_model.add(keras.layers.Dropout(0.5))
final_model.add(keras.layers.Dense(16, activation = "relu"))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.Dense(1, activation = "sigmoid"))

final_model.compile(optimizer = "adam",
            loss = "binary_crossentropy",
            metrics = ["accuracy"])
final_model.fit(set, epochs = 10)
predictions = final_model.predict(set)


input_1 = pd.concat([info_in_order.iloc[:, :-1].reset_index(drop = True), pd.DataFrame(predictions, columns = ["prediction"])], axis = 1)

final_log_reg = LogisticRegression()
final_log_reg.fit(input_1, info_in_order.iloc[:, -1])

Found 485 files belonging to 2 classes.
Epoch 1/10
16/16 [==============================] - 3s 63ms/step - loss: 0.7801 - accuracy: 0.5959
Epoch 2/10
16/16 [==============================] - 1s 62ms/step - loss: 0.6065 - accuracy: 0.6577
Epoch 3/10
16/16 [==============================] - 1s 61ms/step - loss: 0.5436 - accuracy: 0.7361
Epoch 4/10
16/16 [==============================] - 1s 62ms/step - loss: 0.4690 - accuracy: 0.7753
Epoch 5/10
16/16 [==============================] - 1s 62ms/step - loss: 0.4723 - accuracy: 0.7649
Epoch 6/10
16/16 [==============================] - 1s 62ms/step - loss: 0.4319 - accuracy: 0.8062
Epoch 7/10
16/16 [==============================] - 1s 57ms/step - loss: 0.4074 - accuracy: 0.7856
Epoch 8/10
16/16 [==============================] - 1s 56ms/step - loss: 0.3819 - accuracy: 0.8433
Epoch 9/10
16/16 [==============================] - 1s 56ms/step - loss: 0.3395 - accuracy: 0.8433
Epoch 10/10
16/16 [==============================] - 2s 61ms/step


LogisticRegression()

In [ ]:
test_set_01 = manual_label[manual_label["Image"].str.contains("TestSetImage")]
test_set_01_images = keras.utils.image_dataset_from_directory(
    directory = "Data/TestSet01",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_01_images)
final_output = pd.concat([test_set_01.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict(final_output.iloc[:, 1:])

Found 20 files belonging to 1 classes.
1/1 [==============================] - 0s 37ms/step


array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
      dtype=int64)

In [ ]:
np.where(final_model.predict(test_set_01_images) > 0.5, 1, 0).reshape(-1, 1)

1/1 [==============================] - 0s 36ms/step


array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0]])

In [ ]:
test_set_02 = manual_label[manual_label["Image"].str.contains("TestSet02")]
test_set_02_images = keras.utils.image_dataset_from_directory(
    directory = "Data/TestSet02",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_02_images)
final_output = pd.concat([test_set_02.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict_proba(final_output.iloc[:, 1:])

Found 24 files belonging to 1 classes.
1/1 [==============================] - 0s 34ms/step


array([[0.9691662 , 0.0308338 ],
       [0.39763636, 0.60236364],
       [0.03277005, 0.96722995],
       [0.97027196, 0.02972804],
       [0.13146957, 0.86853043],
       [0.96962029, 0.03037971],
       [0.83399647, 0.16600353],
       [0.82481203, 0.17518797],
       [0.56262474, 0.43737526],
       [0.82230087, 0.17769913],
       [0.96923816, 0.03076184],
       [0.98325738, 0.01674262],
       [0.96983809, 0.03016191],
       [0.97096965, 0.02903035],
       [0.96862646, 0.03137354],
       [0.9446948 , 0.0553052 ],
       [0.96958222, 0.03041778],
       [0.38904937, 0.61095063],
       [0.97084052, 0.02915948],
       [0.36140002, 0.63859998],
       [0.96697601, 0.03302399],
       [0.90760735, 0.09239265],
       [0.98926887, 0.01073113],
       [0.8244915 , 0.1755085 ]])